# This is a baseline model showing FastAI library to solve the leaf disease classification problem

In [ ]:
#Check for presence of GPU
!nvidia-smi

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
from fastai.vision.all import *

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Check for shape of training data
train = pd.read_csv('/kaggle/input/cassava-leaf-disease-classification/train.csv')
train.shape

In [ ]:
#Let's see how the training data looks
train.head()

In [ ]:
# Check if there is class imbalance
train['label'].value_counts().plot(kind='bar')

In [ ]:
#Validate if training images directory contain all the files
train_fns = glob.glob('/kaggle/input/cassava-leaf-disease-classification/train_images/*.*')
len(train_fns)

In [ ]:
#Check sample submission file
ss = pd.read_csv('/kaggle/input/cassava-leaf-disease-classification/sample_submission.csv')
ss.head()

# **Fastai dataloaders**

In [ ]:
img2lbl = dict()
for k,v in zip(train.image_id.values,train.label.values):
    img2lbl[k] = v  

In [ ]:
def get_items(_):
    return train.image_id.values

In [ ]:
def get_x(fn):
    img = Image.open('/kaggle/input/cassava-leaf-disease-classification/train_images/' + fn)
    return np.array(img)

In [ ]:
def get_y(fn):
    return img2lbl[fn]

In [ ]:
db = DataBlock(blocks = (ImageBlock, CategoryBlock),
               get_items = get_items,
               get_x = get_x,
               get_y     = get_y,
               splitter  = RandomSplitter(),
               item_tfms = Resize(224),
               batch_tfms= [Normalize.from_stats(*imagenet_stats),*aug_transforms(size=224)]
              )

dls = db.dataloaders('',bs=64)
dls.show_batch()

# Creating a learner using Resnet34 model and train

In [ ]:
learn = cnn_learner(dls,resnet34,CrossEntropyLossFlat,metrics=[accuracy]).to_fp16()

In [ ]:
learn.fine_tune(5)